In [13]:
!pip install PyYAML

In [14]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [15]:
with open('data.yaml',mode='r') as f:
    data_yaml=yaml.load(f,Loader=SafeLoader)

label=data_yaml['names']
print(label)

['person', 'car', 'bus']


In [16]:
#Load Yolo Model
yolo=cv2.dnn.readNetFromONNX('./Model2/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [17]:
#Load the image
img=cv2.imread('./14474517749_600f20185e_c.jpg')
image=img.copy()
row,col,d=image.shape

#get YOLO predictions
#step-1:-convert image into square image(array)
max_rc=max(row,col)
input_image=np.zeros((max_rc,max_rc,3),dtype=np.uint8)
input_image[0:row,0:col]=image
#step2:- get prediction from square array
INPUT_WH_YOLO=640
blob=cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds=yolo.forward()

In [18]:
print(preds.shape)

(1, 25200, 8)


In [19]:
#Non Maximum Suppression
#step-1:- filter detection based on confidence(0.4) and probability(0.25)
detections=preds[0]
boxes=[]
confidences=[]
classes=[]
#width and height of input image
image_w,image_h=input_image.shape[:2]
x_factor=image_w/INPUT_WH_YOLO
y_factor=image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row=detections[i]
    confidence=row[4]
    if confidence>0.4:
        class_score=row[5:].max()
        class_id=row[5:].argmax()
        
        if class_score>0.25:
            cx,cy,w,h=row[0:4]
            left=int((cx-0.5*w)*x_factor)
            top=int((cy-0.5*h)*y_factor)
            width=int(w*x_factor)
            height=int(h*y_factor)
            
            box=np.array([left,top,width,height])
            
            #append values to the list
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np=np.array(boxes).tolist()
confidences_np=np.array(confidences).tolist()

#NMS
index=cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()

In [20]:
index

array([ 3, 16,  1, 13])

In [10]:
#Draw bounding box
for ind in index:
    x,y,w,h=boxes_np[ind]
    bb_conf=int(confidences_np[ind]*100)
    classes_id=classes[ind]
    class_name=label[classes_id]
    
    text=f'{class_name}:{bb_conf}%'
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)
    
    

In [12]:
cv2.imshow('yolo_prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()